In [1]:
import pandas as pd
import re
from datetime import datetime
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import linear_model,metrics
from sklearn.metrics import classification_report

# UDF

In [2]:
#remove outliers
def remove_Outlier_box(values):
    firstq,median,thirdq = values.dropna().quantile([0.25,0.5,0.75])
    IQR = thirdq - firstq
    return ~(((values-thirdq)>1.5*IQR) | ((firstq-values)>1.5*IQR))

In [3]:
def clean_data(df):
    # df = pd.read_csv("rental-large-06-12-2017.csv")
    #total row - 10162

    #remove housetype = residential land as only land is rent out
    #df = df[df['House_Type']!='Residential Land']
    #total row - 10157

    #select sale_or_rent = rent
    df = df[df['Sale_or_Rent'] == 'Rent']
    #total row - 9142

    #extract price number
    df['Price_num'] = df['Price'].map(lambda x:float(
        re.search('[a-zA-Z]*(\d+).*',x.replace(",","")).group(1)))

    #remove price more than 100000
    #df = df[df['Price_num']<100000]
    #total row - 9131

    #extract number from Built_Up_Area and Land_Area and convert it in sqft
    def num_sqft(x):
        if re.search("(\d+) Sq",x.replace(",","")):
            return float(re.search("(\d+) Sq",x.replace(",","")).group(1))
        elif re.search("(.*\d+) Acres",x.replace(",","")):
            return float(re.search("(.*\d+) Acres",x.replace(",","")).group(1))*43560
        else:
            return 0
    df['Built_num_sqft'] = df['Built_Up_Area'].map(num_sqft)
    df['Land_num_sqft'] = df['Land_Area'].map(num_sqft)

    #remove outliers for Built_num_sqft - 1.000138e+06,588060.000000
    #df_noOutlier = df[df['Built_num_sqft'] != df['Built_num_sqft'].max()].copy()
    #total row = 9129
    #df_noOutlier = df_noOutlier[df_noOutlier['Built_num_sqft'] != df_noOutlier['Built_num_sqft'].max()]

    #convert string to datetime
    df['Date_Posted'] = df['Date_Posted'].map(lambda x:datetime.strptime(x,'%Y-%m-%d'))
    #df['Date_Posted'] = df['Date_Posted'].map(lambda x:datetime.strptime(x,'%d %b %Y'))

    #extract year, month and day from datetime
    #df['Year'] = df['Date_Posted'].dt.year
    #df['Month'] = df['Date_Posted'].dt.month
    #df['Day'] = df['Date_Posted'].dt.day
    #from year 2013 til 2017

    #remove multiple listing
    df_nodup = df[~df.duplicated(subset = [i for i in df.columns if i not in ['Date_Posted']])].copy()
    #total row - 8813

    #assign 0 to nan and regex to others
    df_nodup['Bathroom_num'] = df_nodup['Bathroom']\
                                    .map(lambda x:re.sub(r'[a-zA-Z]','',x) if type(x) == str else '0')\
                                    .map(lambda x:int(re.search(r'(\d+)\+*',x.replace("(\d)","")).group(1))
                                         if re.search(r'(\d+)\+*',x.replace("(\d)",""))
                                         else 0)

    #assign 0 to nan to regex to others
    df_nodup['Bedroom_num'] = df_nodup['Bedroom']\
                                .map(lambda x:re.sub(r'[a-zA-Z]','',x) if type(x) == str else '0')\
                                .map(lambda x:int(re.search(r'(\d+)\+*',x.replace("(\d)","")).group(1))
                                if re.search(r'(\d+)\+*',x.replace("(\d)","")) else 0)

    #replace \xa0 with " " for Furnishing
    df_nodup['Furnishing'] = df_nodup['Furnishing'].map(lambda x:x.replace("\xa0", " "))

    #replace \xa0 with " " for Tenure
    df_nodup['Tenure'] = df_nodup['Tenure'].map(lambda x: x.replace("\xa0"," "))

    #replace + or - with '' and treat '' as 0 for Parking
    df_nodup['Parking_num'] = df_nodup['Parking'].map(lambda x:0 
                                                    if str(x).replace("+","").replace("-","")=='' 
                                                    else int(str(x).replace("+","").replace("-","")))

    #categorize house types into four based on NAPIC
    house_dict = {'High Rise': ['Condominium','Service Apartment/Residence','Apartment','Flat','Studio','Penthouse','SOHO'],
    'Terrace':['1 Storey Terrace House','3 Storey Terrace House','Town House'],
    'Semi Detach':['Semi-Detach House','2 Storey Link House','2.5 Storey Link House','1.5 Storey Link House'],
    'Detach':['Bungalow House','Zero-Lot Bungalow']}
    def polish_num(x,cus_dict):
        if str(x) in [j for i in list(cus_dict.values()) for j in i]:
            return [key for key,value in cus_dict.items() if str(x) in value][0]
        else:
            return x
    #df_nodup['House_Type'] = df_nodup['House_Type'].apply(polish_num, cus_dict = house_dict)

    #Compute price per sqft, take land num if not zero else built num
    df_nodup['Price_per_sqft'] = df_nodup.apply(lambda x:x['Price_num']/x['Land_num_sqft']
                                if int(x['Land_num_sqft'])!= 0 else x['Price_num']/x['Built_num_sqft'], axis = 1)

    #Categorization of state as a result of eda
    #little observation for other states other than KL and Selangor
    df_nodup['State_Cat'] = df_nodup['State'].map(lambda x:x if x in ['Selangor','Kuala Lumpur']
                             else 'Others')
    df_final = df_nodup[(df_nodup['Bedroom_num']<10) & (df_nodup['Bathroom_num']<10)
                       &(df_nodup['Built_num_sqft']!=1) & (df_nodup['Land_num_sqft']>=1000)].copy()
    
    return df_final

In [4]:
def mad_outliers(points, thr = 1):
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis = 0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation
    
    #greater than threshold is outliers, recommended by Iglewicz and Hoaglin 
    return modified_z_score<thr

In [5]:
def getSizeCat(size):
    sizeCat_list = []
    for i in size:
        if i<=500:
            sizeCat_list.append('200-500')
        elif i<=800:
            sizeCat_list.append('501-800')
        elif i<=1100:
            sizeCat_list.append('801-1100')
        elif i<=1400:
            sizeCat_list.append('1101-1400')
        elif i<=1700:
            sizeCat_list.append('1401-1700')
        else:
            sizeCat_list.append('more than 1701')
    return sizeCat_list

In [6]:
#Estate source
Estate123_table = pd.read_csv("../data/processed/hs_clean_rent.csv")

#extract year
Estate123_table['year'] = Estate123_table['Date_Posted'].map(lambda x:re.search('(\d{4})',x).group(1))

Estate123_table['month'] = pd.to_datetime(Estate123_table['Date_Posted']).dt.month

#add column of quarter and year-quarter
Estate123_table['quarter'] = ((Estate123_table['month']-1)//3+1)

Estate123_table['year-quarter'] = Estate123_table['year'].astype(str) + '-' + Estate123_table['quarter'].astype(str)

Estate123_table = Estate123_table[Estate123_table['House_Type']
                                .map(lambda x:x in ['Service Apartment/Residence','Condominium',
                                                'Flat','SOHO','Apartment'])].copy()

In [7]:
Estate_train_table_clean = clean_data(Estate123_table)
Estate_train_table_noOutlier = Estate_train_table_clean[remove_Outlier_box(Estate_train_table_clean['Price_num'])].copy()

In [8]:
Estate_train_table_clean['House_Type'].value_counts()

Condominium                    2741
Service Apartment/Residence     242
Apartment                       214
SOHO                             28
Flat                              3
Name: House_Type, dtype: int64

In [189]:
Estate_train_table_clean[Estate_train_table_clean['House_Type'] == 'Service Apartment/Residence']

Address  \
8                                                     -   
19                                                    -   
30                                                    -   
63            Jalan SS16/1, 47500 Subang Jaya, Selangor   
161                                        jalan tengah   
165                                    jalan tun razak    
166                                        Jalan Tengah   
236                                     jalan tun razak   
237                                     jalan tun razak   
380   6 Jalan Taman Seputeh 58000 Kuala Lumpur Malaysia   
393                                                   -   
408                                                   -   
452                                          titiwangsa   
504                                                   -   
509                                                   -   
560                                                   -   
654                                                   -   
781                                 Mid Valley Megamall   
783                          Jalan Pinang, Kuala Lumpur   
806                                                   -   
837                            Mid Valley, Kuala Lumpur   
838    KLCC Menara Pinang Furnished condo for rent, ...   
915                                             A-11-17   
942                                       Taman Seputeh   
948                                                   -   
1017                                      jalan cempaka   
1038                                                  -   
1063  D'Villa Boutique, 5-storey Service Apartment, ...   
1065  Condo Sri Tiara Service-Apartment, KL, Midvall...   
1126             Waldorf Aptm,Sri Hartamas,Kuala Lumpur   
...                                                 ...   
6930                         Metia Residence, Shah Alam   
6972                                         kl sentral   
6980                               Kuala Lumpur Sentral   
7009                               Kuala Lumpur Sentral   
7017            Kota Damansara, Petaling Jaya, Selangor   
7022                                                  -   
7038                                                  -   
7040                                                  -   
7096                                 Jalan Bukit Ceylon   
7097                                 Jalan Bukit Ceylon   
7110                                 jalan bukit ceylon   
7134                                         kl sentral   
7135                               Kuala Lumpur Sentral   
7190                                                  -   
7199                           Jalan Setia Tropika 1/21   
7205                            jalan persiaran tanjung   
7290                                                  -   
7325                              Taman Serdang Perdana   
7461                    Kuala Lumpur City Centre (KLCC)   
7471                                    Jalan Tun Razak   
7484                                   Jalan Tun Razak    
7496                Jalan 1/76 Off Jalan Kampung Pandan   
7502  Central Residence @ Suria Sungai Besi, No 366,...   
7546                               Jalan Harmonium 24/2   
7548                           Jalan Setia Tropika 1/21   
7639                                   jalan masai baru   
7683                                                  -   
7767                     cyberjaya , cyber 5 , cyber 11   
7796                                                  -   
7809  Multimedia 7/Af, off Jalan, Jalan Plumbum 7/10...   

                                     Area Bathroom Bedroom  \
8                              USJ Da Men        2       2   
19                                    NaN        2       3   
30                        Serdang Perdana        2       3   
63                                  SS 16        2       3   
161                          Jalan Tengah        3   2 + 1   
165              

In [190]:
Estate_train_table_clean['Furnishing'].unique()

array(['Fully Furnished', 'None', 'Partially Furnished'], dtype=object)

In [105]:
#Feature engineering
Estate_train_table_noOutlier['Size_Category'] = getSizeCat(Estate_train_table_noOutlier['Built_num_sqft'])

In [197]:
#gather predictors
Estate_train_table_final = Estate_train_table_noOutlier[['Bedroom_num','Price_num','Price_per_sqft',
                                                         'State','Parking_num','Bathroom_num',
                                                        'Size_Category','House_Type','Furnishing']]\
                        .rename(columns = {'Bedroom_num':'num_bedroom',
                                                      'Price_num':'price',
                                      'Price_per_sqft':'Price_persqft',
                                              'Parking_num':'carpark',
                                                'Bathroom_num':'bath',
                                          'Furnishing':'furnishing'}).copy()

#get dummy variables from categorical variables    
noY_list = [i for i in Estate_train_table_final.columns if i!='House_Type']
Estate_CategoricalV = pd.get_dummies(Estate_train_table_final
                                     [[i for i in noY_list 
                                       if Estate_train_table_final[i].dtype == 'object']])
Estate_NumericalV = Estate_train_table_final[[i for i in noY_list if Estate_train_table_final[i].dtype != 'object']]
Estate_MergedV = pd.concat([Estate_NumericalV,Estate_CategoricalV],join_axes= [Estate_NumericalV.index],axis = 1)

In [199]:
#Star Source
Star_table = pd.read_csv("../data/processed/Propwall_Report_ForRent-26-3-18.csv")
Star_table["State"] = Star_table['State'].apply(lambda x: x if x != 'Petaling Jaya' else 'Selangor')

Star_table['Size_Category'] = getSizeCat(Star_table['size'])
Star_table['furnishing'] = Star_table['furnishing'].map(lambda x:{'Fully Furnished':'Fully Furnished',
                                      'Semi-Furnished':'Partially Furnished',
                                      'Unfurnished':'None',
                                      'nan':'None'}[str(x)])

Star_table_final = Star_table[(Star_table['Type'] == 'Condominium')
                             & (Star_table['layout'] != 'Studio')][noY_list].copy()

Star_CategoricalV = pd.get_dummies(Star_table_final
                                     [[i for i in noY_list 
                                       if Star_table_final[i].dtype == 'object']])
Star_NumericalV = Star_table_final[[i for i in noY_list if Star_table_final[i].dtype != 'object']]
Star_MergedV = pd.concat([Star_NumericalV,Star_CategoricalV],join_axes= [Star_NumericalV.index],axis = 1)

In [200]:
#split dataset into training and testing sets
similarX_set = set(Star_MergedV.columns) & set(Estate_MergedV.columns)
train_x, test_x, train_y, test_y = train_test_split(Estate_MergedV[list(similarX_set)],
    Estate_train_table_final['House_Type'], train_size=0.5,random_state=2)

lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg')
lr.fit(train_x, train_y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [201]:
test_y.unique()

array(['Condominium', 'Service Apartment/Residence', 'SOHO', 'Apartment',
       'Flat'], dtype=object)

# Evaluation

In [202]:
print("Multinomial Logistic regression Train Accuracy :: ", metrics.accuracy_score(train_y, lr.predict(train_x)))
print("Multinomial Logistic regression Test Accuracy :: ", metrics.accuracy_score(test_y, lr.predict(test_x)))

Multinomial Logistic regression Train Accuracy ::  0.847043701799
Multinomial Logistic regression Test Accuracy ::  0.847784200385


In [203]:
print(classification_report(test_y, lr.predict(test_x)))

                             precision    recall  f1-score   support

                  Apartment       0.75      0.03      0.06       100
                Condominium       0.85      0.99      0.92      1322
                       Flat       0.00      0.00      0.00         2
                       SOHO       0.33      0.06      0.10        18
Service Apartment/Residence       0.17      0.01      0.02       115

                avg / total       0.79      0.85      0.79      1557



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [113]:
result_table = pd.DataFrame({'realY':pd.Series(test_y.values),
             'predictedY':pd.Series(lr.predict(test_x))})

# Prediction

In [204]:
Star_table_condo_noStudio = Star_table[(Star_table['Type'] == 'Condominium')
                                      & (Star_table['layout'] != 'Studio')].copy()
Star_table_condo_noStudio['Predicted_HouseType'] = list(lr.predict(Star_MergedV[list(similarX_set)]))

In [205]:
Star_table_condo_noStudio['Predicted_HouseType'].value_counts()

Condominium                    498714
Apartment                        7893
SOHO                              323
Service Apartment/Residence        37
Name: Predicted_HouseType, dtype: int64

In [209]:
#Profile of Condo Type
Star_table_condo_Studio = Star_table[(Star_table['Type'] == 'Condominium')
                                      & (Star_table['layout'] == 'Studio')].copy()
Star_table_condo_Studio['Predicted_HouseType'] = 'Studio'
Star_table_condo_all = pd.concat([Star_table_condo_noStudio,Star_table_condo_Studio])

In [215]:
Star_table_condo_all.groupby('Predicted_HouseType')['price'].mean() 

Predicted_HouseType
Apartment                      1450.409223
Condominium                    2361.993909
SOHO                           6627.780186
Service Apartment/Residence    2445.675676
Studio                         1511.572424
Name: price, dtype: float64

In [216]:
Star_table_condo_all.groupby('Predicted_HouseType')['size'].mean() 

Predicted_HouseType
Apartment                      1031.306601
Condominium                    1218.892038
SOHO                           2566.678019
Service Apartment/Residence     999.459459
Studio                          617.913821
Name: size, dtype: float64

# Exploration

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(test_y[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

#Exploration
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc

y = label_binarize(Estate_train_table_final['House_Type'],classes = Estate_train_table_final['House_Type'].unique())

n_classes = y.shape[1]

X_train, X_test, y_train, y_test = train_test_split(Estate_MergedV, y, test_size=.5,
                                                    random_state=0)

# Learn to predict each class against the other
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True,
                                 random_state=0))
y_score = classifier.fit(X_train, y_train).decision_function(X_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()